In [ ]:
# Import

import copy as cp
import os

import networkx as nx
import numpy as np
import pandas as pd

import pickle
import matplotlib.pyplot as plt
import datetime
import time

# Source: https://github.com/eamonustc/geodistance
import geodistance

**Data handling**

In [ ]:
"""
Loads movement data.
"""
def load_data(DATAPATH_VEN,DATAPATH_MOV):
    data_ven = pd.read_csv(DATAPATH_VEN, encoding="utf-8", error_bad_lines=False, warn_bad_lines=False)
    data_mov = pd.read_csv(DATAPATH_MOV, encoding="utf-8", error_bad_lines=False, warn_bad_lines=False)
    
    data_ven = data_ven[(data_ven.lng != "\\\\N") & (data_ven.lat != "\\\\N")]
    
    data_ven.lng = data_ven.lng.astype(float)
    data_ven.lat = data_ven.lat.astype(float)
    
    data_mov.checkins = data_mov.checkins.astype(int)
    
    #print("Data geladen")
    return data_ven, data_mov

"""
Selects venues based on location, and returns venues located within the
specified rectangle.
"""
def crop_venues(data_ven, lon_min, lon_max, lat_min, lat_max):
    data_ven = data_ven[(data_ven.lng>=lon_min) &
                        (data_ven.lng< lon_max) &
                        (data_ven.lat>=lat_min) &
                        (data_ven.lat< lat_max)]
    return data_ven.reset_index(drop=True)

def select_movements_known_venues(data_ven, data_mov):
    ids = pd.DataFrame(data_ven.id)
    data_mov_id1 = pd.merge(ids, data_mov    , how="inner", left_on="id", right_on="venue1").drop(["id"], axis=1)
    data_mov_id2 = pd.merge(ids, data_mov_id1, how="inner", left_on="id", right_on="venue2").drop(["id"], axis=1)
    
    return data_mov_id2.reset_index(drop=True)

"""
Selects movements based on time: month (months) and time of day (tods).
"""
def select_movements_time(data_mov, months=None, tods=None):
    if tods is not None:
        data_mov = data_mov[data_mov.period.isin(tods)]
    
    if months is not None:
        data_mov = data_mov[data_mov.month.isin(months)]
    
    return data_mov.reset_index(drop=True)

"""
Assigns nearest station to all venues.
"""
def find_closest_stations(data_ven):
    categ = ["Train Stations", "Light Rail Stations", "Metro Stations", "Tram Stations"]
    stations = data_ven[data_ven.category.isin(categ)].drop(["name", "category"], axis=1)
    stations_ind = np.array(stations.index)
    
    lv = len(data_ven)
    ls = len(stations)
    data_ven_the = np.array([data_ven.lat.values*np.pi/180, ]*ls).T
    data_ven_phi = np.array([data_ven.lng.values*np.pi/180, ]*ls).T
    stations_the = np.array([stations.lat.values*np.pi/180, ]*lv)
    stations_phi = np.array([stations.lng.values*np.pi/180, ]*lv)
    dthe = data_ven_the-stations_the
    dphi = data_ven_phi-stations_phi
    a = np.square(np.sin(dthe/2))+np.cos(data_ven_the)*np.cos(stations_the)*np.square(np.sin(dphi/2))
    d = 6371*2*np.arcsin(np.sqrt(a))
    mind = np.reshape(np.min(d, axis=1), (-1, 1))
    argmind = np.argmin(d, axis=1)
    argmind_ind = stations_ind[argmind]
    argmind_id = pd.DataFrame(data_ven.iloc[argmind_ind]).reset_index(drop=True)["id"]
    argmind_id.columns = ["argmind_id"]
    
    data_ven_stations = cp.deepcopy(data_ven)
    data_ven_stations["argmind_id"] = argmind_id
    data_ven_stations["mind"] = mind
    return data_ven_stations

"""
Computes the probability of being traversed by car for every trajectory
and multiplies the number of checkins for this trajectory by this probability.
"""
def calc_checkins_auto(data_ven, data_mov):
    data_ven_stations = find_closest_stations(data_ven).set_index("id")
    
    idA = data_mov.venue1.values
    idB = data_mov.venue2.values
    A = data_ven_stations.loc[idA]
    B = data_ven_stations.loc[idB]
    idA_T = A.argmind_id.values
    idB_T = B.argmind_id.values
    A_T = data_ven_stations.loc[idA_T]
    B_T = data_ven_stations.loc[idB_T]
    
    A_the = A.lat.values
    A_phi = A.lng.values
    B_the = B.lat.values
    B_phi = B.lng.values
    dthe = A_the-B_the
    dphi = A_phi-B_phi
    a = np.square(np.sin(dthe/2))+np.cos(A_the)*np.cos(B_the)*np.square(np.sin(dphi/2))
    d = 6371*2*np.arcsin(np.sqrt(a))
    
    A_T_the = A_T.lat.values
    A_T_phi = A_T.lng.values
    B_T_the = B_T.lat.values
    B_T_phi = B_T.lng.values
    dthe_T = A_T_the-B_T_the
    dphi_T = A_T_phi-B_T_phi
    a_T = np.square(np.sin(dthe_T/2))+np.cos(A_T_the)*np.cos(B_T_the)*np.square(np.sin(dphi_T/2))
    d_T = 6371*2*np.arcsin(np.sqrt(a_T))
    
    # Los Angeles
    '''
    alpha = 3.6
    u_walk  = alpha*(-0.2*d)
    u_auto  = alpha*(-0.6-0.025*d)
    u_train = alpha*(-0.2*(A.mind.values+B.mind.values)-0.025*d_T-1.2)
    '''
    # Tokyo
    alpha=2
    u_walk =alpha*(-0.25*d+0.2)
    u_auto =alpha*(-1.85-0.05*d)
    u_train=alpha*(-0.25*(A.mind.values+B.mind.values)-0.05*d_T-1.4)
    
    P_walk  = np.exp(u_walk)/(np.exp(u_walk)+np.exp(u_auto)+np.exp(u_train))
    P_auto  = np.exp(u_auto)/(np.exp(u_walk)+np.exp(u_auto)+np.exp(u_train))
    P_train = np.exp(u_train)/(np.exp(u_walk)+np.exp(u_auto)+np.exp(u_train))
    
    data_mov.checkins *= P_auto
    
    return data_mov

In [ ]:
np.set_printoptions(threshold=20)

"""
Selects the nr movements with the most checkins.
"""
def select_venues_movements_most_checkins(data_ven, data_mov, nr):
    data_mov_n = data_mov.sort_values(by="checkins", ascending=False)[:nr]
    id1_n = pd.DataFrame(data_mov_n.venue1)
    id2_n = pd.DataFrame(data_mov_n.venue2)
    ids_n = pd.concat([    id1_n.rename(index=str, columns={"venue1": "id"}),
                        id2_n.rename(index=str, columns={"venue2": "id"})]).drop_duplicates().reset_index(drop=True)
    data_ven_n = pd.merge(ids_n, data_ven)
    
    return data_ven_n.reset_index(drop=True), data_mov_n

"""
Clusters all remaining venues to the nr venues with the highest checkins, based
on the minimal Euclidean distance. The origin and destination venues are
changed to that of the nearest high-checkin venue, after which duplicates are
summed and deleted.
"""
def cluster(data_ven, data_mov, nr):
    data_ven_n, data_mov_n = select_venues_movements_most_checkins(data_ven, data_mov, nr)
    vals_ven_pos   =   data_ven.drop(["id", "name", "category"], axis=1).values
    vals_ven_pos_n = data_ven_n.drop(["id", "name", "category"], axis=1).values
    ids   = pd.DataFrame(data_ven.id  )
    ids_n = pd.DataFrame(data_ven_n.id)
    n = len(data_ven_n)
    N = len(data_ven)
    
    # Find nearest neighbour
    lat   = np.matrix([vals_ven_pos  [:, 0], ]*n).T
    lat_n = np.matrix([vals_ven_pos_n[:, 0], ]*N)
    lon   = np.matrix([vals_ven_pos  [:, 1], ]*n).T
    lon_n = np.matrix([vals_ven_pos_n[:, 1], ]*N)    
    r_sq = np.square(lat-lat_n)+np.square(lon-lon_n)
    r_argmin = np.ravel(np.argmin(r_sq, axis=1))
    ids_argmin = ids_n.iloc[r_argmin].reset_index(drop=True)
    ids_nn = pd.concat([ids, ids_argmin], axis=1)
    ids_nn.columns = ["id", "id_nn"]
    
    # Add ids_nn to data_ven
    data_ven_nn = pd.merge(ids_nn, data_ven, how="inner", on="id")
    
    # Replace venue1 and venue2 in data_mov by values from the appropriate id
    # of the nn-table
    data_mov_nn = pd.merge(ids_nn, data_mov   , how="inner", left_on="id", right_on="venue2")
    data_mov_nn = data_mov_nn.drop(["id", "venue2"], axis=1).rename(index=str, columns={"id_nn": "venue2"})
    data_mov_nn = pd.merge(ids_nn, data_mov_nn, how="inner", left_on="id", right_on="venue1")
    data_mov_nn = data_mov_nn.drop(["id", "venue1"], axis=1).rename(index=str, columns={"id_nn": "venue1"})
    
    # Sum checkins of duplicates and drop duplicates
    data_mov_nn["checkins_nn"] = data_mov_nn.groupby(["venue1", "venue2", "month", "period"])["checkins"].transform("sum")
    data_mov_nn = data_mov_nn.drop_duplicates(subset=["venue1", "venue2", "month", "period"]).drop(["checkins"], axis=1)
    data_mov_nn = data_mov_nn[data_mov_nn.venue1 != data_mov_nn.venue2]
    
    return data_ven_nn, data_mov_nn


def compute_num_clusters(auto_num_clusters,custom_num_clusters,avg_distance,num_lanes,max_density):
    num_clusters = -1
    if(not(auto_num_clusters)):
        num_clusters = custom_num_clusters
    else:
        num_clusters = int(total_movements / (avg_distance*num_lanes*max_density/2))
    return num_clusters

In [ ]:
def load_datasets(num_movements,num_clusters,datapath_ven,datapath_mov):
    daily_traffic_amount = num_movements 

    # Load and process data
    data_ven, data_mov = load_data(datapath_ven,datapath_mov)
    data_ven = crop_venues(data_ven, 139.704515, 139.861661, 35.594325, 35.726745)
    data_mov = select_movements_known_venues(data_ven, data_mov)
    data_mov = select_movements_time(data_mov, tods=["AFTERNOON"]) 

    # Scale

    data_mov = calc_checkins_auto(data_ven,data_mov)

    # Cluster data
    data_ven_clustered, data_mov_clustered = cluster(data_ven, data_mov, num_clusters)

    # Drop original venue ids, drop duplicates of new ids
    data_ven_clustered = data_ven_clustered.drop('id',axis=1)
    data_ven_new = data_ven_clustered.drop_duplicates(subset="id_nn", keep='first', inplace=False).dropna()

    # Take average checkins per timestep (morning/afternoon/evening/night, so 6 hours)
    # Timestep is currently useless since we already select on afternoon only
    data_mov_clustered['checkins_nn'] = data_mov_clustered.groupby(['venue1', 'venue2'])['checkins_nn'].transform('sum')
    data_mov_clustered['checkins_count_month'] = data_mov_clustered.groupby(['venue1', 'venue2'])['checkins_nn'].transform('count')
    data_mov_clustered['checkins_count_month_time'] = data_mov_clustered.groupby(['venue1', 'venue2', 'month'])['checkins_nn'].transform('count')
    data_mov_clustered = data_mov_clustered.drop_duplicates(subset=("venue1","venue2"), keep='first', inplace=False).dropna()
    data_mov_clustered['checkins_avg_per_timestep'] = data_mov_clustered['checkins_nn'] / data_mov_clustered['checkins_count_month'] / data_mov_clustered['checkins_count_month_time']
    data_mov_clustered['checkins_percentage'] = data_mov_clustered['checkins_avg_per_timestep'] / np.sum(data_mov_clustered['checkins_avg_per_timestep'].values) * daily_traffic_amount
    # Above not really percentage, cause it already multiplies

    # Drop movement intermediate columns
    data_mov_clustered = data_mov_clustered.drop(['checkins_count_month','checkins_count_month_time','checkins_avg_per_timestep'],axis=1)

    # Drop duplicates
    data_mov_new = data_mov_clustered.drop_duplicates(subset=("venue1","venue2"), keep='first', inplace=False).dropna()

    return data_ven_new,data_mov_new


**Graph and Algorithm**

In [ ]:
# Build graph with lat/lon of most visited stuff


def build_graph(shapefile_path,save_graph,num_lanes,max_density,max_speed):

    # Load road network
    # Based on code from:
    # https://gis.stackexchange.com/questions/256955/how-to-load-a-weighed-shapefile-in-networkx
    X=nx.read_shp(shapefile_path).to_undirected()

    coords = {}
    pos = {k: v for k,v in enumerate(X.nodes())}
    G=nx.Graph()
    for nodenum,coord_pair in pos.items():
        lat = coord_pair[1]
        lon = coord_pair[0]
        G.add_node("intersection_"+str(nodenum), lng=lon, lat=lat, node_type='road_intersection')
        coords["intersection_" + str(nodenum)] = (lat,lon)


    l=[set(x) for x in X.edges()] #To speed things up in case of large objects
    edg=[tuple(k for k,v in pos.items() if v in sl) for sl in l] #Map the G.edges start and endpoints onto pos

    for i in range(0,len(edg)):
        tup = edg[i]
        x = tup[0]
        y = tup[1]
        x = "intersection_" + str(x)
        y = "intersection_" + str(y)
        new_tup = (x,y)
        edg[i] = new_tup

    j = 0
    for edge in edg:
        node1 = edge[0]
        node2 = edge[1]
        data1 = G.nodes[node1]
        data2 = G.nodes[node2]
        dist,br = geodistance.distanceHaversine(data1['lat'],data1['lng'],data2['lat'],data2['lng']) # Distance in km
        fee1 = np.random.uniform(low=1,high=10) # Initialize fees randomly
        G.add_edge(node1,
                   node2,
                   distance=dist,
                   fee=fee1,
                   total_weight=dist+fee1,
                   num_lanes=num_lanes,
                   key="road_"+str(j),
                   occupancy=0,
                   travel_time=0,
                   max_velocity=max_speed,
                   max_density=max_density
                  )
        j += 1



    # Add nodes for venues

    for idex,row in data_ven_new.iterrows():
        node_id = row[0] # We'll need this to identify the nodes
        node_lat = float(row[2])
        node_lon = float(row[3])
        G.add_node('venue_'+str(node_id), lng=node_lon, lat=node_lat, key=node_id, node_type='venue') # can't export if pos is defined; do manually in gephi
        # Connect venue to nearest neighbor

        lowest_nodenum = -1
        lowest_dist = float('inf')
        for nodenum,data in coords.items():
            target_lat = data[0]
            target_lon = data[1]
            dist,br = geodistance.distanceHaversine(node_lat,node_lon,target_lat,target_lon)
            if(dist < lowest_dist):
                lowest_nodenum = nodenum
                lowest_dist = dist
        fee1 = np.random.uniform(low=1,high=10)
        G.add_edge("venue_"+str(node_id),
                   lowest_nodenum,
                   distance=lowest_dist,
                   fee=fee1,
                   total_weight=dist+fee1,
                   num_lanes=num_lanes,
                   key="road_"+str(j),
                   occupancy=0,
                   travel_time=0,
                   max_velocity=max_speed,
                   max_density=999999999999
                  )
        j += 1



    # Export for gephi; display there using geo-layout
    if(save_graph):
        nx.write_gexf(G, "graph_shapefile_taipeigen.gexf")
    return G


In [ ]:
# Define objective function

def obj(individual,beta,G,data_mov,min_speed):
    # Step 0: Set fee values for all edges using parameters of the solution
    i = 0
    
    for edge_data in G.edges.data():
        a = edge_data[0]
        b = edge_data[1]
        d = edge_data[2]
        key = d['key']
        dist = d['distance']
        fee = d['fee']
        total_weight = beta*dist+individual[i]
        # when a, b and key are the same, add_edge modifies existing
        G.add_edge(a,b,key=key,fee=individual[i],
                   total_weight=total_weight,occupancy=0) # Reset occupancy to prevent explosion
        i += 1
    

    
    # Step 1: for all clustered origin-destination pairs, find the shortest path using distance+fees
    
    # For each route, maintain tuple of (time,checkins) for both conditions
    travel_times = []
    travel_times_checkins = []
    # For each route, remember path taken, may be tough on memory, but can't do it during
    # other loops due to road-focus. Need occupancy from all routes before computing travel
    # time of specific route.
    paths = []

    for idex,row in data_mov_new.iterrows():
        # Iterate over origin-destination pairs
        
        origin = "venue_" + row[0]
        destination = "venue_" + row[1]
        num_checkins = row[5]
        
        # Find shortest path
        path = nx.dijkstra_path(G,origin,destination,weight='total_weight')
        paths.append(path)
        
        last_node = None

        # Step 2: use total amount of check-ins for those pairs to compute total amount of cars on each road segment of the path
        # Update occupancy by num_checkins
        
        
        for node_name in path:
            if(last_node != None):
                d = G.get_edge_data(last_node,node_name)
                key = d['key']
                occupancy = d['occupancy']
                max_dens = d['max_density']
                num_lanes = d['num_lanes']
                dist = d['distance']
                if(((occupancy+num_checkins)/dist) > (max_dens*num_lanes)):
                    G.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins,fee=999999999999)
                else:
                    G.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins)
            last_node = node_name
            


    
    # Step 3: use traffic formulas to compute the average time spent on each road segment
    total_time_all_roads = 0
    i = 0
    for edge_data in G.edges.data():
        a = edge_data[0]
        b = edge_data[1]
        d = edge_data[2]
        num_lanes = d['num_lanes']
        key = d['key']
        occ = d['occupancy']
        dist = d['distance']
        max_vel = d['max_velocity']
        max_dens = d['max_density'] * num_lanes
        opt_dens = max_dens/2
        max_f = max_vel * opt_dens
        
        
        if(occ>0):
            dens = occ/dist         
            fp = (max_f/(opt_dens**2)) * max(0,dens*(2*(max_dens/2)-dens))
            avg_speed = min(max_vel,fp/dens)        
            average_travel_time = dist/max(avg_speed,min_speed)
           
        else:
            average_travel_time = 0
            
        # Step 4: multiply time by amount of cars to get a total time spent
        total_travel_time = average_travel_time * occ
        G.add_edge(a,b,key=key,travel_time=average_travel_time)
        
        # Step 5: sum for all roads
        
        total_time_all_roads += total_travel_time
        i += 1

    # Step 6: Compute distribution of times
        

    i = 0
    for idex,row in data_mov_new.iterrows():
        # Iterate over origin-destination pairs
        
        origin = "venue_" + row[0]
        destination = "venue_" + row[1]
        num_checkins = row[5]
        path = paths[i]
        i += 1
        
        current_time = 0
        last_node = None
        for node_name in path:
            if(last_node != None):
                d = G.get_edge_data(last_node,node_name)
                travel_time = d['travel_time']
                current_time += travel_time                
            last_node = node_name
        travel_times.append(current_time)
        travel_times_checkins.append(num_checkins) # Only need this once

    
    x_cost = []
    i = 0
    for time in travel_times:
        for j in range(0,int(travel_times_checkins[i])):
            x_cost.append(time)
        i+=1
    return sum(x_cost)




In [ ]:
# Standard GA


def run_GA(G,data_mov_new,iteration_budget,population_size,mutation_rate,sigma,beta,min_speed):

    genome_length = len(G.edges)

    # Global variables

    best_fitness_per_iteration = []
    best_individual = np.zeros(genome_length)
    best_fitness = float('inf') # Objective is to minimize

    # Initialize population randomly

    population = np.random.uniform(0,1,size=(population_size,genome_length)) # Rows are individuals, columns are fees

    # Start main loop
    for iter in range(1,iteration_budget+1):
        working_population = np.zeros((2*population_size,genome_length+1))

        # Recombination

        i = 0
        while(i<population_size):
            parent1 = population[i,:]
            parent2 = population[i+1,:]
            crossover_point = np.random.randint(1,genome_length-1)
            offspring1 = np.concatenate([parent1[0:crossover_point],parent2[crossover_point:genome_length]])
            offspring2 = np.concatenate([parent2[0:crossover_point],parent1[crossover_point:genome_length]])
            working_population[i,0:genome_length] = offspring1
            working_population[i+1,0:genome_length] = offspring2
            working_population[i+population_size,0:genome_length] = parent1
            working_population[i+1+population_size,0:genome_length] = parent2
            i += 2


        # Mutation

        for i in range(0,population_size):
            for j in range(0,genome_length):
                if(np.random.uniform() < mutation_rate):
                    working_population[i,j] = max(0,working_population[i,j] + np.random.normal(loc=0,scale=sigma))

        # Store best individual and its fitness

        for i in range(0,2*population_size):
            a = datetime.datetime.now()
            working_population[i,genome_length] = obj(working_population[i,0:genome_length],beta,
                                                      G,data_mov_new,min_speed) # Not +1 cause 0 indexing
            b = datetime.datetime.now()
            delta = b - a

        ranking = working_population[working_population[:,genome_length].argsort()]
        local_top_individual = ranking[0,0:genome_length]
        local_top_fitness = ranking[0,genome_length]
        best_fitness_per_iteration.append(local_top_fitness)
        if(local_top_fitness < best_fitness):
            best_fitness = local_top_fitness
            best_individual = local_top_individual

        # Select and update population

        population = ranking[0:population_size,0:genome_length]

    # Uncomment to plot fitness improvement

    #plt.plot(best_fitness_per_iteration)
    #plt.ylabel('Fitness')
    #plt.xlabel('Iteration')
    #plt.show()
    return best_individual, best_fitness_per_iteration





In [ ]:
# Copy of objective function, used to save output graph with a list of parameters 

# Define objective function

def obj_eval(individual,best_fitness,beta,G,data_mov,experiment_name,histogram_bins,sigma,num_clusters,min_speed):
    # Step 0: Set fee values for all edges using parameters of the solution
    i = 0
    
    for edge_data in G.edges.data():
        a = edge_data[0]
        b = edge_data[1]
        d = edge_data[2]
        key = d['key']
        dist = d['distance']
        fee = d['fee']
        total_weight = beta*dist+individual[i]
        # when a, b and key are the same, add_edge modifies existing
        G.add_edge(a,b,key=key,fee=individual[i],
                   total_weight=total_weight,occupancy=0) # Reset occupancy to prevent explosion
        i += 1
    
    G2 = G.copy()
    
    # Step 1: for all clustered origin-destination pairs, find the shortest path using distance+fees
    
    # For each route, maintain tuple of (time,checkins) for both conditions
    travel_times = []
    travel_times_nocost = []
    travel_times_checkins = []
    # For each route, remember path taken, may be tough on memory, but can't do it during
    # other loops due to road-focus. Need occupancy from all routes before computing travel
    # time of specific route.
    paths = []
    paths_nocost = []
    
    
    for idex,row in data_mov_new.iterrows():
        # Iterate over origin-destination pairs
        
        origin = "venue_" + row[0]
        destination = "venue_" + row[1]
        num_checkins = row[5]
        
        # Find shortest path
        path = nx.dijkstra_path(G,origin,destination,weight='total_weight')
        paths.append(path)
        path_nocost = nx.dijkstra_path(G2,origin,destination,weight='distance')
        paths_nocost.append(path_nocost)
        
        last_node = None

        # Step 2: use total amount of check-ins for those pairs to compute total amount of cars on each road segment of the path
        # Update occupancy by num_checkins
        
        
        
        for node_name in path:
            if(last_node != None):
                d = G.get_edge_data(last_node,node_name)
                key = d['key']
                occupancy = d['occupancy']
                max_dens = d['max_density']
                num_lanes = d['num_lanes']
                dist = d['distance']
                if(((occupancy+num_checkins)/dist) > (max_dens*num_lanes)):
                    G.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins,fee=999999999999)
                else:
                    G.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins)
            last_node = node_name
            
        last_node = None
        for node_name in path_nocost:
            if(last_node != None):
                d = G2.get_edge_data(last_node,node_name)
                key = d['key']
                occupancy = d['occupancy']
                max_dens = d['max_density']
                num_lanes = d['num_lanes']
                dist = d['distance']
                if(((occupancy+num_checkins)/dist) > (max_dens*num_lanes)):
                    G2.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins,fee=999999999999)
                else:
                    G2.add_edge(last_node,node_name,key=key,occupancy=occupancy+num_checkins)
            last_node = node_name
    

    
    # Step 3: use traffic formulas to compute the average time spent on each road segment
    total_time_all_roads = 0
    i = 0
    for edge_data in G.edges.data():
        a = edge_data[0]
        b = edge_data[1]
        d = edge_data[2]
        num_lanes = d['num_lanes']
        key = d['key']
        occ = d['occupancy']
        dist = d['distance']
        max_vel = d['max_velocity']
        max_dens = d['max_density'] * num_lanes
        opt_dens = max_dens/2
        max_f = max_vel * opt_dens
        
        
        if(occ>0):
            dens = occ/dist         
            fp = (max_f/(opt_dens**2)) * max(0,dens*(2*(max_dens/2)-dens))
            avg_speed = min(max_vel,fp/dens)        
            average_travel_time = dist/max(avg_speed,min_speed)
           
        else:
            average_travel_time = 0
            
        # Step 4: multiply time by amount of cars to get a total time spent
        total_travel_time = average_travel_time * occ
        G.add_edge(a,b,key=key,travel_time=average_travel_time)
        
        # Step 5: sum for all roads
        
        total_time_all_roads += total_travel_time
        i += 1

    # Step 6: Compute distribution of times
        

    i = 0
    # With cost
    for idex,row in data_mov_new.iterrows():
        # Iterate over origin-destination pairs
        
        origin = "venue_" + row[0]
        destination = "venue_" + row[1]
        num_checkins = row[5]
        path = paths[i]
        i += 1
        
        current_time = 0
        last_node = None
        for node_name in path:
            if(last_node != None):
                d = G.get_edge_data(last_node,node_name)
                travel_time = d['travel_time']
                current_time += travel_time                
            last_node = node_name
        travel_times.append(current_time)
        travel_times_checkins.append(num_checkins) # Only need this once

        
        
    # Repeat steps 3-6 Without cost for comparison
    
    # Step 3: use traffic formulas to compute the average time spent on each road segment
    total_time_all_roads = 0
    i = 0
    for edge_data in G2.edges.data():
        a = edge_data[0]
        b = edge_data[1]
        d = edge_data[2]
        num_lanes = d['num_lanes']
        key = d['key']
        occ = d['occupancy']
        dist = d['distance']
        max_vel = d['max_velocity']
        max_dens = d['max_density'] * num_lanes
        opt_dens = max_dens/2
        max_f = max_vel * opt_dens
        
        
        if(occ>0):
            dens = occ/dist         
            fp = (max_f/(opt_dens**2)) * max(0,dens*(2*(max_dens/2)-dens))
            avg_speed = min(max_vel,fp/dens)        
            average_travel_time = dist/max(avg_speed,min_speed) 
        else:
            average_travel_time = 0
            
  
        # Step 4: multiply time by amount of cars to get a total time spent
        total_travel_time = average_travel_time * occ
        G2.add_edge(a,b,key=key,travel_time=average_travel_time)
        
        # Step 5: sum for all roads
        total_time_all_roads += total_travel_time
        i += 1

        
    # Step 6: Compute distribution of times
              
    i = 0
    # Without cost
    for idex,row in data_mov_new.iterrows():
        # Iterate over origin-destination pairs
        
        origin = "venue_" + row[0]
        destination = "venue_" + row[1]
        num_checkins = row[5]
        path = paths_nocost[i]
        i += 1
        
        current_time = 0
        last_node = None
        for node_name in path:
            if(last_node != None):
                d = G2.get_edge_data(last_node,node_name)
                travel_time = d['travel_time']
                current_time += travel_time                
            last_node = node_name
        travel_times_nocost.append(current_time)

    
    # Compute statistical measures
    
    measures = [min(travel_times),max(travel_times),np.mean(travel_times),np.median(travel_times),np.std(travel_times)]
    measures_nocost = [min(travel_times_nocost),max(travel_times_nocost),np.mean(travel_times_nocost),
                       np.median(travel_times_nocost),np.std(travel_times_nocost)]
    
    # Histogram preparation, since hist doesn't seem to work with explicitly specifying frequencies (checkins)
    x_cost = []
    x_nocost = []
    i = 0
    for time in travel_times:
        for j in range(0,int(travel_times_checkins[i])):
            x_cost.append(time)
        i+=1
    i = 0
    for time in travel_times_nocost:
        for j in range(0,int(travel_times_checkins[i])):
            x_nocost.append(time)
        i+=1
        
    # Step 7: Save stuff

    # Check if directory exists
    if(not(os.path.isdir("results/" + experiment_name))):
        os.mkdir("results/" + experiment_name)
    
    # Find number
    it = 0
    for file in os.listdir("results/" + experiment_name + "/"):
        if file.endswith(".gexf"):
            it += 1
    
    # Save graph 
    for e in G.edges.data():
        d = e[2]
        d['occupancy'] = int(d['occupancy'])
        d['fee'] = float(d['fee'])
        d['total_weight'] = float(d['total_weight'])
    nx.write_gexf(G, "results/" + experiment_name + "/run" + str(it) + "_graph.gexf")
    
    # Save algorithm progress figure
    x = range(1,iteration_budget+1)
    default_fitness = obj(np.zeros(len(G.edges)),beta,G,data_mov_new,min_speed)
    random_fitness = 0
    for i in range(0,5): # Hardcoded as it's not central, can be parameterised
        r = abs(np.random.randn(len(G.edges))) * sigma
        random_fitness += obj(r,beta,G,data_mov_new,min_speed)
    random_fitness = random_fitness / (i+1)
    default_fitness_improvement = abs((default_fitness - best_fitness) / default_fitness * 100)
    random_fitness_improvement = abs((random_fitness - best_fitness) / random_fitness * 100)
    d = [default_fitness for i in range(1,iteration_budget+1)]
    r = [random_fitness for i in range(1,iteration_budget+1)]
    
    plt.plot(x,best_fitness_per_iteration)
    plt.ylabel('Fitness')
    plt.xlabel('Iteration')
    plt.plot(x,d)
    plt.plot(x,r)
    plt.legend(['GA','No external costs','Random external costs'])
    plt.savefig("results/" + experiment_name + "/run" + str(it) + "_optimisation_progress.png")
    plt.clf()
    
    # Save time distribution figures
    num_bins = histogram_bins
    
    max_val_x = max(max(x_cost),max(x_nocost))
    n, bins, patches = plt.hist(x_cost, num_bins, facecolor='blue', alpha=0.5)
    plt.xlabel("Time spent on the road in hours, using external costs")
    plt.ylabel("Frequency")
    plt.xlim(left=0,right=max_val_x)
    plt.savefig("results/" + experiment_name + "/run" + str(it) + "_hist.png")
    stored_ylim = plt.gca().get_ylim()
    plt.clf()
    
    n, bins, patches = plt.hist(x_nocost, bins=bins, facecolor='blue', alpha=0.5)
    plt.xlabel("Time spent on the road in hours, without external costs")
    plt.ylabel("Frequency")
    plt.xlim(left=0,right=max_val_x)
    plt.ylim(stored_ylim)
    plt.savefig("results/" + experiment_name + "/run" + str(it) + "_hist_nocost.png")
    plt.clf()
    
    
    
    
    # Roads hist
    edge_times = []
    for edge_data in G.edges.data():
        d = edge_data[2]
        edge_times.append(d['travel_time'])
        
    
    n, bins, patches = plt.hist(edge_times, num_bins, facecolor='blue', alpha=0.5)
    plt.xlabel("Time spent on roads")
    plt.ylabel("Frequency")
    plt.savefig("results/" + experiment_name + "/run" + str(it) + "_hist_roads.png")
    plt.clf()
    
    
    
    # Save parameter settings
    
    if(it==0):
        # Write header if file doesn't exist yet
        with open("results/" + experiment_name + "/settings_" + experiment_name + ".csv",'w') as fp:
            s = "run_num,default_fitness,random_fitness,best_fitness,default_fitness_improvement,\
            random_fitness_improvement,[best_fitness_per_iteration],iteration_budget,mutation_rate,sigma,beta,\
            [min_time,max_time,mean_time,median_time,std_time],\
            [min_time_nocost,max_time_nocost,mean_time_nocost,median_time_nocost,std_time_nocost],num_clusters\n"
            fp.write(s)
    
    with open("results/" + experiment_name + "/settings_" + experiment_name + ".csv",'a') as fp:
        param_string = str(it) + ","+ str(default_fitness) + "," + str(random_fitness) + "," + \
                        str(best_fitness) + "," + str(default_fitness_improvement) + "," + \
                        str(random_fitness_improvement) + "," + str(best_fitness_per_iteration) + "," + \
                        str(iteration_budget) + "," + str(mutation_rate) + "," + str(sigma) + "," + \
                        str(beta) + "," + str(measures) + "," + str(measures_nocost) + "," + str(num_clusters) + "\n"
        fp.write(param_string)
        




In [ ]:
# Controller function for experiments

# Experiment parameters (default in comments)
experiment_name = "newobj_movements_25000"

shapefile_path = 'shapefiles/groads/tokyo_smallest.shp' # 'shapefiles/groads/tokyo_smallest.shp'
num_runs = 1 # 5
histogram_bins = 10 # 15
save_graph = False # False

# Data handling parameters
total_movements = 25000 # 25000
avg_distance = 1.0948129267976516 # Computed beforehand
datapath = "foursquare/" # "foursquare/"
auto_num_clusters = False # True
custom_num_clusters = 38 # 100
datapath_ven = datapath + "venues/Tokyo_venue_info.csv" # "venues/Tokyo_venue_info.csv"
datapath_mov = datapath + "movements/Tokyo_movements.csv" # "movements/Tokyo_movements.csv"

# Graph parameters

num_lanes = 2*2
max_density = 300
max_speed = 100


# GA parameters
iteration_budget = 30 # 30
population_size = 20 # 20
mutation_rate = 0.2 # 0.2
sigma = 4 # 4
beta = 2 # 2

# Obj parameters

min_speed = 15



# Compute num_clusters

num_clusters = compute_num_clusters(auto_num_clusters,custom_num_clusters,avg_distance,num_lanes,max_density)
print("Number of clusters: " + str(num_clusters))
print("Projected runtime for " + str(num_runs) + " runs: " + str(int(num_clusters/200*15000/60*num_runs)) + " minutes.")
print("Started running at: " + str(time.time()))
print("\n\n")

# Load movement data

print("Loading and processing movement data...")
data_ven_new,data_mov_new = load_datasets(total_movements,num_clusters,datapath_ven,datapath_mov)
print("...done.")


# Load graph data

print("Loading spatial data and building graph...")
G = build_graph(shapefile_path,save_graph,num_lanes,max_density,max_speed)
print("...done.")



# Run experiments

print("Starting experiments...\n")
start_time = time.time()
for i in range(0,num_runs):
    print("Running experiment " + str(i+1) + "/" + str(num_runs))
    iteration_start_time = time.time()
    G = build_graph(shapefile_path,False,num_lanes,max_density,max_speed) # Since it takes little computational power, just to prevent carry-over effects
    best_individual, best_fitness_per_iteration = run_GA(G,data_mov_new,iteration_budget,population_size,mutation_rate,sigma,beta,min_speed)
    obj_eval(best_individual,best_fitness_per_iteration[-1],beta,G,data_mov_new,experiment_name,histogram_bins,sigma,num_clusters,min_speed)
    iteration_end_time = time.time()
    print("...done. Experiment runtime: " + str(iteration_end_time - iteration_start_time))

end_time = time.time()
print("\nExperiments concluded. Total runtime: " + str(end_time - start_time))








In [ ]:
# Function for running experiments with random total_movements

# Experiment parameters (default in comments)
experiment_name = "newgen_total_movement_opt_autocluster"

shapefile_path = 'shapefiles/groads/tokyo_smallest.shp' # 'shapefiles/groads/tokyo_smallest.shp'
num_runs = 20 # 5
histogram_bins = 15 # 15
save_graph = False # False

# Data handling parameters
max_total_movements = 150000 # 12000000 (12 million)
min_total_movements = 25000
num_clusters = 100 # 100
avg_distance = 1.0948129267976516 # Computed beforehand
datapath = "foursquare/" # "foursquare/"
auto_num_clusters = True # True
custom_num_clusters = 100 # 100
datapath_ven = datapath + "venues/Tokyo_venue_info.csv" # "venues/Tokyo_venue_info.csv"
datapath_mov = datapath + "movements/Tokyo_movements.csv" # "movements/Tokyo_movements.csv"

# Graph parameters

num_lanes = 2*2
max_density = 300
max_speed = 100


# GA parameters
iteration_budget = 30 # 30
population_size = 20 # 20
mutation_rate = 0.2 # 0.2
sigma = 4 # 4
beta = 2 # 2

# Obj parameters

min_speed = 15



# Run experiments

#data_string = ""

print("Starting experiments...\n")
start_time = time.time()
for i in range(0,num_runs):
    print("Running experiment " + str(i+1) + "/" + str(num_runs))
    iteration_start_time = time.time()
    total_movements = int(np.random.uniform(min_total_movements,max_total_movements))
    print("Total movements: " + str(total_movements))
    num_clusters = compute_num_clusters(auto_num_clusters,custom_num_clusters,avg_distance,num_lanes,max_density)
    print("Number of clusters: " + str(num_clusters))
    data_ven_new,data_mov_new = load_datasets(total_movements,num_clusters,datapath_ven,datapath_mov)
    G = build_graph(shapefile_path,False,num_lanes,max_density,max_speed)
    default_fitness = obj(np.zeros(len(G.edges)),beta,G,data_mov_new,min_speed)
    best_individual, best_fitness_per_iteration = run_GA(G,data_mov_new,iteration_budget,population_size,mutation_rate,sigma,beta,min_speed)
    improvement = abs((best_fitness_per_iteration[-1] - default_fitness) / default_fitness)
    obj_eval(best_individual,best_fitness_per_iteration[-1],beta,G,data_mov_new,experiment_name,histogram_bins,sigma,num_clusters,min_speed)
    #data_string = (data_string + str(total_movements) + "," + str(improvement) + "\n")
    data_string = (str(total_movements) + "," + str(improvement) + "\n")
    with open("results/newgen_random_total_movements_autocluster.csv",'a') as fp:
        fp.write(data_string)
    iteration_end_time = time.time()
    print("...done. Experiment runtime: " + str(iteration_end_time - iteration_start_time))

end_time = time.time()
print("\nExperiments concluded and results saved. Total runtime: " + str(end_time - start_time))

In [ ]:
# Scatterplot of total_movements and improvement percentage

d = pd.read_csv("results/newgen_random_total_movements_autocluster.csv",header=None)
x = d.as_matrix()[:,0]
y = d.as_matrix()[:,1] * 100

plt.scatter(x,y)
plt.xlim(0,180000)
plt.xlabel("Total amount of movements")
plt.ylabel("Improvement percentage")


